In [1]:
!pip install transformers==4.46.2 peft==0.13.2 accelerate==1.1.1 trl==0.12.1 bitsandbytes==0.45.2 datasets==3.1.0 huggingface-hub==0.26.2 safetensors==0.4.5 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.8/434.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [8]:
# Base model (from Hugging Face Hub)
model_name = "NousResearch/Llama-2-7b-chat-hf"

# Instruction dataset
dataset_name = "TejasKoti/ProjectDataset"

# Fine-tuned model output directory
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

################################################################################
# bitsandbytes config
################################################################################
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################
output_dir = "./results"
num_train_epochs = 5
fp16 = True
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

################################################################################
# Dataset prep
################################################################################
dataset = load_dataset(dataset_name, split="train")
print("Dataset columns:", dataset.column_names)

# ⚠️ Replace this with the actual column name from your dataset
text_column = "train"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# LoRA config
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Training args
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Tokenization
def preprocess_function(examples):
    return tokenizer(examples[text_column], padding="max_length", truncation=True)

processed_dataset = dataset.map(preprocess_function, batched=True)

# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=processed_dataset,
    args=training_arguments,
    peft_config=peft_config,
    data_collator=data_collator,
)

# Train
trainer.train()

Dataset columns: ['train']


model-00001-of-00002.safetensors:  15%|#5        | 1.54G/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Map:   0%|          | 0/3266 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


In [9]:
# Save model + tokenizer
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

print(f"✅ Fine-tuned model saved to {new_model}")

✅ Fine-tuned model saved to Llama-2-7b-chat-finetune


In [10]:
# Silence warnings
logging.set_verbosity(logging.CRITICAL)

# Quick test
prompt = "Explain labour laws"
pipe = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print("Generated output:", result[0]['generated_text'])

Generated output: <s>[INST] Explain labour laws [/INST]  Labour laws, also known as employment laws or labor laws, are regulations that govern the relationship between employers and employees. everybody involved in the workplace. These laws aim to protect workers' rights, ensure fair treatment, and promote safe working conditions. Here are some key aspects of labour laws:

1. Minimum Wage: Labour laws set a minimum wage that employers must pay their employees. This wage is intended to ensure that workers are paid a fair and living wage for their work.
2. Overtime Pay: Many labour laws require employers to pay employees overtime pay for working more than a certain number of hours in a day or week. This is intended to provide workers with additional compensation for their extra work.
3. Working Hours: Labour laws set limits on the number of hours an employee can work in a day or week. These limits are intended to ensure that workers have enough time to rest and recover from their work.
4

In [11]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_name = "NousResearch/Llama-2-7b-chat-hf"
adapter_path = "./Llama-2-7b-chat-finetune"
save_path = "./lawyer-llama-merged"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype="auto",
    device_map="auto"
)

# Attach adapter (let PEFT handle the mapping)
model = PeftModel.from_pretrained(base_model, adapter_path, is_trainable=False)

# Merge LoRA
model = model.merge_and_unload()

# Save tokenizer + merged model
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

print("✅ Merged and saved at", save_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Merged and saved at ./lawyer-llama-merged


In [21]:
from huggingface_hub import login

# 1. Login (you'll paste your HF token here)
!huggingface-cli login

# 2. Push model and tokenizer
save_path = "./lawyer-llama-merged"   # the folder you saved merged model + tokenizer

model.push_to_hub("TejasKoti/Llama-2-7b-lawyer-chat", commit_message="Merged LoRA with base model")
tokenizer.push_to_hub("TejasKoti/Llama-2-7b-lawyer-chat", commit_message="Tokenizer for merged model")



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/TejasKoti/Llama-2-7b-lawyer-chat/commit/7f730590d6af8758fee0f110ae5a19c582b8a1bc', commit_message='Tokenizer for merged model', commit_description='', oid='7f730590d6af8758fee0f110ae5a19c582b8a1bc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/TejasKoti/Llama-2-7b-lawyer-chat', endpoint='https://huggingface.co', repo_type='model', repo_id='TejasKoti/Llama-2-7b-lawyer-chat'), pr_revision=None, pr_num=None)